<a href="https://colab.research.google.com/github/Ragul-SL/Non-Invasive-BloodPressure/blob/main/Estimate_BP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import json
from os.path import join
import matplotlib.pyplot as plt
from scipy import signal

%matplotlib inline

In [7]:
input_path = '/content/Cuff-less Non-invasive Blood Pressure Estimation Data Set'
num_patients = 26

In [8]:
with open(join(input_path, '1.json'), 'r') as f:
    data = json.load(f)

In [9]:
data_keys = ['data_PPG', 'data_ECG', 'data_PCG', 'data_FSR', 'data_BP']

In [10]:
{k: len(data[k]) for k in data_keys}

{'data_PPG': 280000,
 'data_ECG': 280000,
 'data_PCG': 280000,
 'data_FSR': 280000,
 'data_BP': 4}

In [11]:
data['data_BP']

[{'SBP': 141, 'DBP': 82},
 {'SBP': 134, 'DBP': 73},
 {'SBP': 125, 'DBP': 70},
 {'SBP': 118, 'DBP': 57}]